In [ ]:
#Q1 build_hash
#dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

In [6]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
#@Q2. Indexing the data
#index

In [7]:
import requests 
from elasticsearch import Elasticsearch
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
es = Elasticsearch("http://localhost:9200")
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        res = es.index(index="my-index", document=doc)

In [9]:
#Q3
import requests 
from elasticsearch import Elasticsearch
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()
es = Elasticsearch("http://localhost:9200")
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        res = es.index(index="my-index", document=doc)
        

In [ ]:
#Q3
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

response = es.search(
    index="my-index",
    body={
        "query": {
            "multi_match": {
                "query": "How do execute a command on a Kubernetes pod?",
                "fields": ["question^4", "text"],
                "type": "best_fields"
            }
        }
    }
)

top_score = response["hits"]["hits"][0]["_score"]
print("Top ranking score:", top_score)

/tmp/ipykernel_2020/3472592042.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


Top ranking score: 44.56891


In [12]:
#Q4
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

response = es.search(
    index="my-index",
    body={
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": "How do copy a file to a Docker container?",
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "match": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
)
if len(response["hits"]["hits"]) >= 3:
    print("3rd question:", response["hits"]["hits"][2]["_source"]["question"])
else:
    print("Less than 3 results found.")

3rd question: How do I debug a docker container?


/tmp/ipykernel_2020/4136988158.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(


In [16]:
#Q5
question = "How do I debug a docker container?"
text = """Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)"""

context_template = """
Q: {question}
A: {text}
""".strip()

context = context_template.format(
    question=question,
    text=text
)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

final_prompt = prompt_template.format(
    question="How do I execute a command in a running docker container?",
    context=context
)

print("Prompt length:", len(final_prompt))

Prompt length: 602


In [ ]:
#Q6
import os
import tiktoken

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-XXXXXX"  # Replace with your actual API key

context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join([
    context_template.format(
        question=hit["_source"]["question"],
        text=hit["_source"]["text"]
    )
    for hit in response["hits"]["hits"][:3]
])

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

final_prompt = prompt_template.format(
    question="How do I execute a command in a running docker container?",
    context=context
)

encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(final_prompt))
print("Number of tokens in the prompt:", num_tokens)

Number of tokens in the prompt: 298
